In [8]:
import cPickle as pickle
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from copy import deepcopy
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [9]:
info = load_pickle("Anaheim, California features.pkl")


IOError: [Errno 2] No such file or directory: 'Anaheim, California features.pkl'

In [10]:
def accuracy(svm, X, y):
    preds = svm.predict(X)
    wrong = 0
    dist = 0
    for (p,l) in zip(preds,y):
        if int(p) != int(l):
            wrong += 1
            dist += abs(l-p)/2.0
    print "correct predictions: %f%%"%(wrong/float(len(y)))
    print "average error: %f"%(dist/wrong)

In [11]:
def toBinary(s):
    return 0 if s == None else (1 if s.lower() == 'yes' else 0)

# Input is a list of dictionarys mapping features to values (defined in pickle) 
def transform_to_XY(info):
    X = []
    Y = []
    feature_names = [i for i in info[0].keys()]
    total_vals = {}
    for x in feature_names:
        total_vals[x] = set()
    
    to_ignore = set(['name', 'rating', 'location', 'categories', 'url', 'parking', 'Best Nights','Good For'])
    
    for res in info:
        for k,v in res.items():
            if k not in to_ignore: # Some features are of list type or already floats, or not primitive types
                total_vals[k].add(v)
    
    #print total_vals
    feature_value_map = {}
    for k,v in total_vals.items():
        z = 0
        feature_value_map[k] = {}
        for value in v:
            feature_value_map[k][value] = z
            z += 1
            
    features = deepcopy(info)
    for b in features:
        b["nones"] = sum([1 if b[f] == None else 0 for f in b])
        for f in to_ignore:
            b.pop(f, None)
        for f in b:
            if f not in to_ignore and f in feature_value_map and b[f] in feature_value_map[f]:
                if f not in ["review_count",'number_of_images']:
                    b[f] = feature_value_map[f][b[f]]
    features = [[d[f] for f in sorted(d.keys())] for d in features]
    return (features, [f["rating"]*2 for f in info])

(X,y) = transform_to_XY(info[:900])
(Xp, yp) = transform_to_XY(info[900:])
clf = svm.SVC(kernel='sigmoid', tol=0.001, verbose=True)
clf.fit(X, y)  
accuracy(clf, Xp, yp)
print clf.score(Xp,yp)

NameError: name 'info' is not defined

In [12]:
clf2 = RandomForestClassifier(n_estimators=10)
clf2 = clf2.fit(X, y)
accuracy(clf2, Xp, yp)

NameError: name 'X' is not defined

In [13]:
clf3 = LogisticRegression()
clf3 = clf3.fit(X, y)
accuracy(clf3, Xp, yp)

NameError: name 'X' is not defined